---

In [1]:
import sys
sys.path.append("../../../../")
from ML.utils.mlflow_flow import set_tracking, quick_log_and_register

from ML.utils.utils_yose import load_data, make_features, build_preprocessor

from sklearn.pipeline import Pipeline
from sklearn.base import clone

import numpy as np

from warnings import filterwarnings

filterwarnings("ignore")
set_tracking("http://127.0.0.1:5000")

---

In [2]:
df_train, df_test = load_data()
X = df_train.drop(["Id","SalePrice"], axis=1)
y = np.log1p(df_train["SalePrice"])
X_test = df_test.drop("Id", axis=1)
X = make_features(X)
X_test = make_features(X_test)
pre = build_preprocessor(X)

In [3]:
from sklearn.ensemble import RandomForestRegressor

model = Pipeline([
    ("preprocessor", clone(pre)),
    ("model", RandomForestRegressor())
])

model.fit(X, y)

y_pred = model.predict(X_test)

In [4]:
params = getattr(model, "steps")[1][1].get_params()

try:
    MODEL_URI, VERSION = quick_log_and_register(
        experiment="HousePrices-Competition",
        run_name="RandomForest!",
        model=model,
        X=X, y=y, X_test=X_test,
        params=params,
        metrics = {"r2": model.score(X, y), "rmse": np.sqrt(model.score(X, y))},
        model_name="HousePrices_RandomForest",
        tags={"task":"regression", "dataset":"Kaggle-HousePrices"}
    )

    print("URI:", MODEL_URI, "| version:", VERSION)

except Exception as e:
    print("Error:", e)

Subiendo modelo...
Modelo subido.
URI: runs:/927667fbdc2845ff98d4515eb618ec42/model | version: 1


---

---

---

---